In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence

import time

from Levenshtein import distance as lev
from sklearn.feature_extraction.text import CountVectorizer

import torch


2023-07-24 08:13:15.592720: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 08:13:15.720130: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-24 08:13:15.720149: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-24 08:13:16.383530: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Read (pass)words from file
Initially we only read the first 100 passwords

If you want to read all the lines, use .readlines()

In [2]:
def read_passwords(filename, comp_nr_lines, nr_lines):
    passwords = []
    comparison_pw = []

    with open(filename, "r") as file:
        for i in range(comp_nr_lines):
            comparison_pw.append(next(file).strip())
        for i in range(nr_lines):
            passwords.append(next(file).strip())
            
    return passwords, comparison_pw
        

In [3]:
def read_words(filename, nr_lines):
    words = []
    with open(filename, "r") as file:
        for i in range(nr_lines):
            words.append(next(file).strip().split("\t")[1])
    return words

In [4]:
def create_feature_labels(passwords, words):
    return np.concatenate((np.ones(len(passwords)), np.zeros(len(words))), axis=0)

In [5]:
def create_nn_labels(passwords, words):
    return torch.hstack((torch.concatenate((torch.ones((1,len(passwords))), torch.zeros(1,len(words))), axis=0),
                         torch.concatenate((torch.zeros((1,len(passwords))), torch.ones((1,len(words)))), axis=0)))



Passwords are 1 and nonpassword strings are 0

# Feature selection
Build features


## Levenshtein
To speed up this calculation, it might be worth it to create an implementation of Levenshtein that stops as soon as a distance larger than the current lowest is found. This lowers the average time complexity (but not worst case)

In [6]:
def calculate_levenshtein_distance(word, passwords):
    low = 42000
    for pw in passwords:
        d = lev(word, pw, score_cutoff=low-1)
        if d < low:
            low = d
        if low == 0:
            return low
    return low

## Other features

In [7]:

def count_non_repeating(text):
    '''
    Remove repeating letters from a string
    E.g. aaabbbccccccaaa becomes abca
    
    text: input text
    return: text without repeating letters
    '''
    count = 0
    for i, c in enumerate(text):
        if i ==0 or c != text[i-1]:
            count += 1
    return count



def counts(word, levenshtein=False):
    alpha_lower = 0
    alpha_upper = 0
    numeric = 0
    special = 0
    s = ""
    for c in word:
        if c.islower():
            alpha_lower += 1
            s += 'L'
        elif c.isupper():
            alpha_upper += 1
            s += 'U'
        elif c.isnumeric():
            numeric += 1
            s += 'N'
        else:
            special += 1
            s += 'S'
    length = len(word)
    char_sets = bool(alpha_lower) + bool(alpha_upper) + bool(numeric) + bool(special)
    if levenshtein: 
        lev_d = calculate_levenshtein_distance(word, comparison_pw)
        return [length, alpha_lower, alpha_lower/length, alpha_upper, alpha_upper/length, numeric, numeric/length, special, special/length, char_sets, count_non_repeating(s), lev_d]
    else:
        return [length, alpha_lower, alpha_lower/length, alpha_upper, alpha_upper/length, numeric, numeric/length, special, special/length, char_sets, count_non_repeating(s)]
    


## ngram features

In [8]:
#def word2ngrams(word, n=2):
#    return [' ' + word[:n-1]] + ["".join(j) for j in zip(*[word[i:] for i in range(n)])] + [word[-(n-1):] + ' ']

In [9]:
#print(word2ngrams("test", 2))


# Machine Learning shizzle

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [11]:
def print_errors(y_test, y_pred):
    for i in range(len(y_test)):
        if y_test[i] != y_pred[i]:
            print(words_test[i])
            print(y_test[i])
            print('')

In [12]:
def average_confusion_matrix(matrices):
    cumul_matrices = np.zeros(matrices[0].shape)
    for matrix in matrices:
        cumul_matrices = np.add(cumul_matrices, matrix)
    return cumul_matrices / len(matrices)
        

In [30]:
def run_ML_model(total, labels, nr_splits = 10, model = "decisiontree"):
    
    accuracies = []
    confusion_matrices = []
    
    kf = KFold(n_splits = nr_splits, shuffle=True, random_state = 42)
    for train, test in kf.split(total):
        
        X_train_tot, X_test_tot, y_train, y_test = total[train], total[test], labels[train], labels[test]
        X_train = [features for _, features in X_train_tot]
        words_train = [words for words, _ in X_train_tot]

        X_test = [features for _, features in X_test_tot]
        words_test = [words for words, _ in X_test_tot]


        # pick model
        if model == "decisiontree":
            clf = tree.DecisionTreeClassifier()
        elif model == "randomforest":
            clf = RandomForestClassifier(max_depth = 10, random_state=0)
        elif model == "GaussianNB":
            clf = GaussianNB()
        elif model == "MultinomialNB":
            clf = MultinomialNB()
        else:
            raise ValueError('Unknown model type supplied: "{}". Please specify a different model type'.format(model))

        clf.fit(X_train, y_train)
    
    
        y_pred = clf.predict(X_test)
        accuracies.append(accuracy_score(y_pred, y_test))
        confusion_matrices.append(confusion_matrix(y_test, y_pred))
    
    
    print("Accuracy score: {}".format(np.mean(accuracies)))

    print("\nConfusion matrix:")
    print(average_confusion_matrix(confusion_matrices))
    

# Neural networks

In [14]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

import time


In [15]:
n_epochs = 5
lr = 0.01
n_folds = 5
lstm_input_size = 32
hidden_state_size = 256
n_layers = 2
dropout = 0.125
bidirectional = True
batch_size = 30
num_sequence_layers = 2
output_dim = 11                       # !!!!!!!!!!!!!!!!!!!!!!!!
num_time_steps = 4000                 # !!!!!!!!!!!!!!
rnn_type = 'LSTM'

In [16]:
class Bi_RNN(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=11, num_layers=2, rnn_type='LSTM'):
        super(Bi_RNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        #Define the initial linear hidden layer
        self.init_linear = nn.Linear(self.input_dim, self.input_dim)

        # Define the LSTM layer
        self.lstm = eval('nn.' + rnn_type)(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=bidirectional)

        # Define the output layer
        self.linear1 = nn.Linear(512,128)
        self.linear2 = nn.Linear(128,8)
        self.linear3 = nn.Linear(8,2)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        
        
        
        # Creating PackedSequence
        #packing = nn.utils.rnn.pad_sequence(input)
        #                                                  !!!!!!!!!!!!!!!!!!!!!!!!!!!
        # Insert packing stuff
        
        
        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size ,hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)
        
        
        # Unpacking PackedSequence
        #                                                  !!!!!!!!!!!!!!!!!!!!!!!!!!!
        # Insert packing stuff
        
        
        # Something with Permute???????                    !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        
        # Tanh
        nn.Tanh(input)
        
        # MaxPool 1D
        nn.MaxPool1d()
        
        # Tanh
        nn.Tanh(input)
        
        # Squeeze
        
        # First linear layer
        input = self.linear1(input)
        
        # Dropout
        
        # Second linear layer
        input = self.linear2(input)
        
        # Dropout
        
        # Third linear layer
        output = self.linear3(input)
        
        return output


In [17]:
class Dataset_Sequential(Dataset):
    def __init__(self, input, output):
        self.embedding = nn.Embedding(100,32)
        self.input = self.embedding(input)
        self.output = output
        

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        x = self.input[idx]
        y = self.output[idx]
        # Embed the input in numbers rather than characters
        x = self.embedding(x)
        x = torch.tensor(x, dtype=torch.float)
        y = torch.tensor(y, dtype=torch.float)
        return x, y

class Dataset_Sequential_test(Dataset):
    def __init__(self, input):
        self.embedding = nn.Embedding(100,32)
        self.input = self.embedding(input)
        
        
    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        x = self.input[idx]
        # Embed the input in numbers rather than characters
        x = self.embedding(x)
        x = torch.tensor(x, dtype=torch.str)
        return x

In [18]:
def load_data():
    test = Dataset_Sequential_test(X_test)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [19]:
m = nn.MaxPool1d(3, stride = 2)
input = torch.randn(3, 4, 5)
output = m(input)
print(input)
print(output)

tensor([[[-1.4172e+00, -7.8747e-01, -1.1008e+00,  4.1487e-01,  1.7343e+00],
         [ 1.7072e+00, -2.6369e-01, -1.0030e+00, -3.0641e-01, -1.0493e+00],
         [-7.6069e-01,  2.5529e+00, -3.9879e-01,  5.7225e-01, -2.6085e-01],
         [-4.3744e-01,  3.9818e-01, -8.8142e-01, -6.7287e-02, -7.2837e-01]],

        [[-7.4815e-01, -9.4549e-01, -2.4928e-01,  3.3547e-01, -8.6195e-01],
         [-8.9873e-01, -8.4740e-01,  1.0274e+00,  1.3147e+00, -1.0529e+00],
         [ 1.2640e-01,  1.2569e-01, -6.1100e-01,  1.6612e+00, -1.8537e-01],
         [ 2.6297e+00, -1.0318e+00,  4.5603e-02, -7.7397e-01,  5.0445e-01]],

        [[-9.3154e-01, -8.4231e-01,  1.0362e+00, -9.2376e-01,  7.3705e-01],
         [-2.4756e+00, -1.2276e+00, -1.0093e+00, -8.9960e-01,  1.3997e-01],
         [ 8.1091e-01, -1.3730e+00,  5.9778e-01,  5.6640e-04,  2.7450e-01],
         [-1.6370e+00,  1.3457e+00, -8.6471e-01,  1.4560e+00,  5.4191e-01]]])
tensor([[[-0.7875,  1.7343],
         [ 1.7072, -0.3064],
         [ 2.5529,  0.57

In [44]:
def run_LSTM(words_train, y_train):
    #Iterate through folds

    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    local_val_score = 0
    models = {}
    
    nr_classes = 2
    print(len(words_train))
    print(y_train.shape)

    k=0 #initialize fold number
    for tr_idx, val_idx in kfold.split(words_train, torch.transpose(y_train, 0,1)):

        print('starting fold', k)
        k += 1

        print(6*'#', 'splitting and reshaping the data')
        words_train = np.array(words_train).flatten()
        train_input = words_train[tr_idx]
        train_target = y_train[:,tr_idx]
        val_input = words_train[val_idx]
        val_target = y_train[:,val_idx]

        print(6*'#', 'Loading')
        train = Dataset_Sequential(train_input, train_target)
        valid = Dataset_Sequential(val_input, val_target)
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

        #Build tensor data for torch
        train_preds = np.zeros((int(train_input.shape[0] * 2)))
        val_preds = np.zeros((int(val_input.shape[0] * 2)))
        best_val_preds = np.zeros((int(val_input.shape[0] * 2)))
        train_targets = np.zeros((int(train_input.shape[0] * 2)))
        avg_losses_f = []
        avg_val_losses_f = []

        #Define loss function
        loss_fn = torch.nn.CrossEntropyLoss()

        #Build model, initialize weights and define optimizer
        model = Bi_RNN(lstm_input_size, hidden_state_size, batch_size=batch_size, output_dim=output_dim, num_layers=num_sequence_layers, rnn_type=rnn_type)  # (input_dim, hidden_state_size, batch_size, output_dim, num_seq_layers, rnn_type)
        model = model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)  # Using Adam optimizer
        #scheduler = ReduceLROnPlateau(optimizer, 'min', patience=150, factor=0.1, min_lr=1e-8)  # Using ReduceLROnPlateau schedule
        temp_val_loss = 9999999999
        reached_val_score = 0

        #Iterate through epochs
        for epoch in range(n_epochs):
            start_time = time.time()

            #Train
            model.train()
            avg_loss = 0.
            for i, (x_batch, y_batch) in enumerate(train_loader):
                x_batch = x_batch.view(-1, num_time_steps, lstm_input_size)
                y_batch = y_batch.view(-1, num_time_steps, output_dim)
                optimizer.zero_grad()
                y_pred = model(x_batch.cuda())
                loss = loss_fn(y_pred.cpu(), y_batch)
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)
                pred = F.softmax(y_pred, 2).detach().cpu().numpy().argmax(axis=-1)
                train_preds[i * batch_size * train_input.shape[1]:(i + 1) * batch_size * train_input.shape[1]] = pred.reshape((-1))
                train_targets[i * batch_size * train_input.shape[1]:(i + 1) * batch_size * train_input.shape[1]] = y_batch.detach().cpu().numpy().argmax(axis=2).reshape((-1))
                del y_pred, loss, x_batch, y_batch, pred

            #Evaluate
            model.eval()
            avg_val_loss = 0.
            for i, (x_batch, y_batch) in enumerate(valid_loader):
                x_batch = x_batch.view(-1, num_time_steps, lstm_input_size)
                y_batch = y_batch.view(-1, num_time_steps, output_dim)
                y_pred = model(x_batch.cuda()).detach()
                avg_val_loss += loss_fn(y_pred.cpu(), y_batch).item() / len(valid_loader)
                pred = F.softmax(y_pred, 2).detach().cpu().numpy().argmax(axis=-1)
                val_preds[i * batch_size * val_input.shape[1]:(i + 1) * batch_size * val_input.shape[1]] = pred.reshape((-1))
                del y_pred, x_batch, y_batch, pred
            if avg_val_loss < temp_val_loss:
                temp_val_loss = avg_val_loss

            #Calculate F1-score
            train_score = f1_score(train_targets, train_preds, average='macro')
            val_score = f1_score(val_target.argmax(axis=2).reshape((-1)), val_preds, average='macro')

            #Print output of epoch
            elapsed_time = time.time() - start_time
            scheduler.step(avg_val_loss)
            if epoch%10 == 0:
                print('Epoch {}/{} \t loss={:.4f} \t train_f1={:.4f} \t val_loss={:.4f} \t val_f1={:.4f} \t time={:.2f}s'.format(epoch + 1, n_epochs, avg_loss, train_score, avg_val_loss, val_score, elapsed_time))

            if val_score > reached_val_score:
                reached_val_score = val_score
                best_model = copy.deepcopy(model.state_dict())
                best_val_preds = copy.deepcopy(val_preds)

        #Calculate F1-score of the fold
        val_score_fold = f1_score(val_target.argmax(axis=2).reshape((-1)), best_val_preds, average='macro')

        #Save the fold's model in a dictionary
        models[k] = best_model

        #Print F1-score of the fold
        print("BEST VALIDATION SCORE (F1): ", val_score_fold)
        local_val_score += (1/n_folds) * val_score_fold

    #Print final average k-fold CV F1-score
    print("Final Score ", local_val_score)

# Run experiments
The following code allows you to run experiments

In [21]:
pw_filename = "10-million-password-list-top-1000000.txt"
text_filename = "eng_news_2020_1M-words.txt"
comp_nr_lines = 10000
nr_lines = 10000

In [22]:
# Load data
passwords, comparison_pw = read_passwords(pw_filename, comp_nr_lines, nr_lines)
words = read_words(text_filename, nr_lines)
#print(words)
print("Nr of passwords: {}\nNr of words: {}".format(len(passwords), len(words)))

all_words = passwords + words


Nr of passwords: 10000
Nr of words: 10000


## Feature-based models

We have the following feature settings

- base_features: simple feature set such as the length of the word and amount of characters from different character sets
- levenshtein: calculate the Levensthein distance to most common passwords
- ngrams: ngram of characters as features
- ngram_range: if ngrams is true, this setting determines which ngrams are to be taken into account

In [23]:
base_features = True
levenshtein = False
ngrams = False
ngram_range = (1, 2)

In [24]:
# replace next line as soon as we have code that reads in the text file
#text = "Previous research has shown that language and culture influence what passwords look like. Li et al. (2014) compared Chinese and English passwords, and found several differences. First of all, Chinese speakers prefer to use digits while English speakers prefer letters, especially lowercase letters; around 50% of the Chinese passwords were digits only. Besides that, all users use patterns that they are familiar with. In the case of Chinese speakers, this is pinyin words, while in the case of English speakers, these are English words. Finally, they also find that users tend to use date formats that are customary in their culture (e.g. 0104 or 0401 for April 1st). Wang et al. (2019), who also compared Chinese and English passwords, add to this that there are difference in the letter distributions of password groups. For example, Chinese passwords use the letter q more often than English passwords do, likely because q is a common letter in pinyin, but quite rare in English. These distributions do not exactly match the distributions of the languages completely. For example, they found that the letters l and w occurred more often in passwords than in general Chinese pinyin. They argue that this is likely because they are part of the very popular Chinese names Li and Wang. Furthermore, Wang et al. (2019) add that around 11% of Chinese passwords contain a pinyin name (compared to 4% in their English passwords dataset) and about 31% include a 4+ digit date in their password, of which 17% include a 6 digit date. Note that these percentages are likely overestimations as it is not always possible to determine of a number sequence whether it is a date or refers to something else. All these numbers are significantly higher than in the English password set. Phone numbers are also possible passwords. De Tweede Kamerleden waren het er snel over eens: als ze onderzoek wilden doen naar beïnvloeding van moskeeën, dan móésten ze Lorenzo Vidino wel uitnodigen. De gelauwerde extremisme-deskundige uit de Verenigde Staten had er boeken over geschreven, parlementen voorgelicht, regeringen geadviseerd. Dus leek het een logische keuze voor de Kamercommissie die in 2019 onderzoek deed naar buitenlandse moskeefinanciering, om Vidino te horen als expert. Het werd een „heel interessant gesprek”, herinnert een van de commissieleden zich, die anoniem wil blijven omdat de zitting achter gesloten deuren plaatsvond. „Hij schetste een gedetailleerd beeld van de Nederlandse vertakkingen van de Moslimbroederschap.” De islamitische beweging zou hier volgens Vidino „verdeeldheid” zaaien. Hij kon meerdere Nederlandse moskeeën bij naam noemen die onder invloed zouden staan van de Moslimbroeders en wist zelfs de exacte hoogte van ontvangen giften uit Qatar."
#words = text_to_word_sequence(text, lower=False)

# Run feature-based models
labels = create_feature_labels(passwords, words)

labels_nn = create_nn_labels(passwords, words)
print(labels_nn)


tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.]])


In [32]:
start_time = time.time()

features = [counts(pw, levenshtein=levenshtein) for pw in passwords]
features_word = [counts(word, levenshtein=levenshtein) for word in words]
features = np.concatenate((features, features_word), axis=0)


if ngrams: 
    vectorizer = CountVectorizer(analyzer = 'char', lowercase = False, ngram_range = ngram_range)
    ngram_features = vectorizer.fit_transform(all_words)
    # CountVectorizer returns a sparse matrix. This needs to be converted into a dense matrix in order to be able to concatenate it.
    features = np.concatenate((np.array(features), ngram_features.toarray()), axis=1)

# link features and words
total = np.array(list(zip(all_words, features)))

run_ML_model(total, labels, model="randomforest")

print("--- %s seconds ---" % (time.time() - start_time))


/tmp/ipykernel_21274/1647070275.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  total = np.array(list(zip(all_words, features)))


Accuracy score: 0.8042

Confusion matrix:
[[811.8 188.2]
 [203.4 796.6]]
--- 4.2651872634887695 seconds ---


In [45]:
# Run neural network model
run_LSTM(all_words, labels_nn)

20000
torch.Size([2, 20000])
starting fold 0
###### splitting and reshaping the data
###### Loading


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not numpy.ndarray